### Data Exploration
The recipes dataset contains 522,517 recipes from 312 different categories. This dataset provides information about each recipe like cooking times, servings, ingredients, nutrition, instructions, and more.

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("../dataset/recipes.csv")

In [6]:
dataset.head(2)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."


In [7]:
dataset.shape

(522517, 28)

In [13]:
dataset.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [9]:
dataset.isnull().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                       17237
RecipeIngredientQuantities         3
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

### Data Preprocessing

In [4]:
features = dataset[['RecipeIngredientQuantities', 'RecipeIngredientParts','RecipeInstructions']]

In [5]:
features.head(3)

,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions
0,"c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...","c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,"c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...","c(""Soak saffron in warm milk for 5 minutes and..."
2,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...","c(""Into a 1 quart Jar with tight fitting lid, ..."


In [6]:
full_text = features.loc[0, 'RecipeIngredientParts']
print(full_text)

c("blueberries", "granulated sugar", "vanilla yogurt", "lemon juice")


In [8]:
# Converting from R-style list to Python List
def to_python_list(r_list_string):
    if(isinstance(r_list_string, str)):
        return r_list_string.strip('c()').replace('"', '').split(', ')
    return []

In [9]:
test = to_python_list(full_text)
print(test)

['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']


In [11]:
features.loc[:, 'RecipeIngredientParts'] = features['RecipeIngredientParts'].apply(to_python_list)

In [14]:
features.loc[:, 'RecipeInstructions'] = features['RecipeInstructions'].apply(to_python_list)
features.loc[:, 'RecipeIngredientQuantities'] = features['RecipeIngredientQuantities'].apply(to_python_list)

In [15]:
features.head(2)

,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions
0,"[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...","[Toss 2 cups berries with sugar., Let stand fo..."
1,"[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...","[saffron, milk, hot green chili peppers, onion...",[Soak saffron in warm milk for 5 minutes and p...
